In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [10]:
week11_link = 'http://www.footballlocks.com/nfl_odds_week_11.shtml#Closing%20NFL%20Odds%20Week%201,%202007'

#get html text
week11_html = requests.get(week11_link).text

soup = BeautifulSoup(week11_html, 'html.parser')

#want just table row data
tr = soup.find_all("tr")

len(tr)

#start of table
print(tr[51])

#last entry in table
print(tr[263])

<tr>
<td>11/16 8:25 ET</td>
<td>At Pittsburgh</td>
<td>-7</td>
<td>Tennessee</td>
<td>44</td>
<td>-$330 +$270</td>
</tr>
<tr>
<td>11/20 8:30 ET</td>
<td>At Jacksonville</td>
<td>-3.5</td>
<td>NY Giants</td>
<td>38.5</td>
<td>-$200 +$170</td>
</tr>


In [11]:
#want to 264 because it does not include last one
table = tr[51:264]

type(table[-1])

type(table)

string_table = [str(i) for i in table]

type(string_table[0])

week11_df = pd.DataFrame(string_table)

week11_df.columns = ['all_data']

week11_df.head()

,all_data
0,<tr>\n<td>11/16 8:25 ET</td>\n<td>At Pittsburg...
1,<tr>\n<td>11/19 1:00 ET</td>\n<td>Detroit</td>...
2,<tr>\n<td>11/19 1:00 ET</td>\n<td>Kansas City<...
3,<tr>\n<td>11/19 1:00 ET</td>\n<td>Tampa Bay</t...
4,<tr>\n<td>11/19 1:00 ET</td>\n<td>Baltimore</t...


In [12]:
#want to clean the date data

dates = [x.split('td')[1] for x in week11_df['all_data']]

cleaned_dates = []
for x in dates:
    str1 = x.replace(">", "").replace("<", "")
    final_str = str1[:-1]
    cleaned_dates.append(final_str)

week11_df['date'] = cleaned_dates

In [13]:
#pull out favorites

week11_df['cleaned_favorites'] = week11_df['all_data'].map(lambda x: x.split('td')[3].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create underdog column

week11_df['cleaned_underdogs'] = week11_df['all_data'].map(lambda x: x.split('td')[7].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create spread column
week11_df['cleaned_spread'] = week11_df['all_data'].map(lambda x: x.split('td')[5].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create over_under column
week11_df['cleaned_over_under'] = week11_df['all_data'].map(lambda x: x.split('td')[9].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create money_odds column and clean data
money_odds = []
for x in week11_df['all_data']:
    try:
        money_odds.append(x.split('td')[11].replace('<', '').replace('>', '').replace('/', ''))
    except:
        money_odds.append(np.nan)
        
week11_df['money_odds'] = money_odds

#input week number for each game
week11_df['week_number'] = '11'

In [14]:
#want to input year for each game. on lines we want to decrease the count by one, len(week1_df.all_data[x].split('td')) = 3
year = 2018
year_list = []
for x in week11_df['all_data']:
    if len(x.split('td')) > 3:
        year_list.append(year)
    else:
        year_list.append(year)
        year -= 1

week11_df['year'] = year_list

In [15]:
#create a final dataframe for week 11
week11_final = week11_df[['date', 'year', 'week_number', 'cleaned_favorites', 'cleaned_underdogs', 'cleaned_spread', 
                        'cleaned_over_under', 'money_odds']]
week11_final.head()

,date,year,week_number,cleaned_favorites,cleaned_underdogs,cleaned_spread,cleaned_over_under,money_odds
0,11/16 8:25 ET,2018,11,At Pittsburgh,Tennessee,-7,44,-$330 +$270
1,11/19 1:00 ET,2018,11,Detroit,At Chicago,-3,40.5,-$170 +$150
2,11/19 1:00 ET,2018,11,Kansas City,At NY Giants,-10,45,-$475 +$375
3,11/19 1:00 ET,2018,11,Tampa Bay,At Miami,-1.5,44,-$125 +$105
4,11/19 1:00 ET,2018,11,Baltimore,At Green Bay,-2.5,38.5,-$135 +$115


In [16]:
#week11_final to pickle
week11_final.to_pickle('week11_final.pkl')